In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.postprocessor import SimilarityPostprocessor, KeywordNodePostprocessor
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.response_synthesizers import TreeSummarize
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.query_engine import RetrieverQueryEngine
import os
from dotenv import load_dotenv

load_dotenv()
# Configure settings
Settings.llm = Groq(model="llama-3.1-8b-instant", api_key=os.getenv("GROQ_API_KEY"))
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

resource module not available on Windows


e:\LLama-Index\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
e:\LLama-Index\venv\lib\site-packages\transformers\utils\hub.py:106: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
# Load and index documents
documents = SimpleDirectoryReader("medical_papers").load_data()
index = VectorStoreIndex.from_documents(documents)

# Create hybrid retriever
vector_retriever = index.as_retriever(similarity_top_k=10)
bm25_retriever = BM25Retriever.from_defaults(index=index, similarity_top_k=5)



In [5]:
class HybridRetriever:
    def __init__(self, vector_retriever, bm25_retriever):
        self.vector_retriever = vector_retriever
        self.bm25_retriever = bm25_retriever

    def retrieve(self, query):
        vector_nodes = self.vector_retriever.retrieve(query)
        bm25_nodes = self.bm25_retriever.retrieve(query)
        all_nodes = vector_nodes + bm25_nodes
        return sorted(all_nodes, key=lambda n: n.score, reverse=True)[:15]



In [6]:
# Create query engine
query_engine = RetrieverQueryEngine.from_args(
    retriever=HybridRetriever(vector_retriever, bm25_retriever),
    node_postprocessors=[
        SimilarityPostprocessor(similarity_cutoff=0.3),
        KeywordNodePostprocessor(
            required_keywords=["treatment", "study", "patient"],
            exclude_keywords=["animal", "in vitro"]
        )
    ],
    response_mode="tree_summarize"
)


In [7]:
# Test query
response = query_engine.query("What are the treatments for diabetes insipidus?")
print("Answer:", response)
print("Sources:")
for node in response.source_nodes:
    print(f"- {node.text[:200]}... (Score: {node.score:.2f})")

Answer: The treatments for diabetes insipidus include the use of synthetic analogues of vasopressin, such as desmopressin and DDAVP, which can be administered orally, intranasally, or subcutaneously to help regulate fluid balance and reduce urine production. In addition to medication, patients may be advised to drink plenty of fluids to help dilute their urine and reduce the risk of dehydration. Some patients may also need to follow a specific diet or fluid intake plan to help manage their condition. Surgery may be an option for some patients, particularly those with pituitary or suprasellar tumors that are causing the condition. In infants and young children, treatment may involve the use of intranasal desmopressin or subcutaneous DDAVP, and in some cases, a combination of medication and fluid management may be necessary.
Sources:
- Acute suppression of plasma 
vasopressin and thirst after drinking in hypernatremic humans. 
Am J Physiol 1987; 252:R1138–42. https://doi.org/10.1152/
ajp

In [8]:
# Query interface
while True:
    query = input("\nEnter your medical question (or 'exit' to quit): ")
    if query.lower() == "exit":
        break

    response = query_engine.query(query)
    if not response.source_nodes:
        print("No relevant information found. Please try rephrasing your question.")
    else:
        print(f"\nAnswer: {response}\n")
        print("Sources:")
        for node in response.source_nodes:
            if node.score > 0.7:  # Only show high-confidence sources
                print(f"- {node.text[:200]}... (Score: {node.score:.2f})")


Answer: The management of diabetes insipidus in adults and pediatric patients differs due to variations in underlying causes, severity, and response to treatment. Pediatric patients require more intensive monitoring and careful dose titration to prevent hyponatremia, particularly in the first 24-48 hours postoperatively. They also require more frequent and aggressive management of the condition due to their smaller body size and higher fluid requirements.

In contrast, adults do not require the same level of intensive monitoring, and their treatment approach is similar to that in young adults. However, elderly adults face a higher risk of developing hyponatremia, primarily when intranasal DDAVP is used, due to abnormalities of osmoregulation of thirst and fluid intake, as well as increased renal sensitivity to DDAVP.

Additionally, the dosage of DDAVP may need to be adjusted in pediatric patients, and they require education about the features of water intoxication and the hazards of e

: 